## Setting up colab environment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/My Drive/Colab Notebooks/w266_final/project_re"

/content/drive/.shortcut-targets-by-id/1I3W7Z7rz_YfsjBjX7z4zTzGbr-iKTrmD/w266_final/project_re


In [3]:
!pip install transformers
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 1.3MB 12.1MB/s 
     |████████████████████████████████| 890kB 45.1MB/s 
     |████████████████████████████████| 1.1MB 24.6MB/s 
     |████████████████████████████████| 2.9MB 59.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=cdd476cdb370e0226d45c1c3d80ac3e385268c88cf1078f5e931cde7612043bd
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 133kB 11.4MB/s 
     |████████████████████████████████| 133kB 10.1MB/s 
     |████████████████████████████████| 6.9MB 21.1MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
ERROR: botocore 1.19.25 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [4]:
%reload_ext autoreload
%autoreload
%matplotlib inline
import os, json,glob,sys,io
from types import SimpleNamespace
from experiment import run_model
from eval import calculate_stats, eval_model
import pickle
from datetime import datetime 
import torch
from torch import nn
from transformers import BertTokenizer
from model.MedClinical import Biobert_fc 
from sklearn.metrics import classification_report, f1_score
from util.tools  import load_config
from util.DataLoader  import *
from pathlib import Path, PureWindowsPath, PurePosixPath
import traceback
from matplotlib import pyplot as plt
import pandas as pd

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"

In [6]:
config_folder = "config/"
config = load_config(config_folder)

In [7]:
dataprocessor = MultiClassificationProcessor()
test_dataloader, dev_data_len, dev_num_labels, dev_num_train_optimization_steps, all_dev_label_ids = dataprocessor.get_data_loader(config,source='test')    

INFO:root:***** Running training *****
INFO:root:  Num examples = 50010
INFO:root:  Batch size = 24
INFO:root:  Num steps = 20830


In [8]:
def save_missed_cases_to_file(file_name, preds, label_ids, inputs):
    tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
    missed_cases = []
    all_cases = []
    for i in range(0,len(label_ids)):
        all_cases.append([ file_name, preds[i],  label_ids[i] , " ". join (tokenizer.convert_ids_to_tokens(inputs[i])) ])
        if label_ids[i] !=  preds[i]:
            missed_cases.append([ file_name, preds[i],  label_ids[i] , " ". join (tokenizer.convert_ids_to_tokens(inputs[i])) ])
    #Save into a file
    missed_cases_file = config.programsettings.REPORTS_DIR +'missed_test_results_' + str(datetime.now()).replace(":", "_").replace(".", "_") + ".pkl"
    all_cases_file = config.programsettings.REPORTS_DIR +'all_test_results_' + str(datetime.now()).replace(":", "_").replace(".", "_") + ".pkl"
    
    with open(missed_cases_file, "wb") as f:
        pickle.dump(missed_cases, f)  
        
    with open(all_cases_file, "wb") as f:
        pickle.dump(all_cases, f)  
        
    

In [9]:
models_folder_name = config.programsettings.OUTPUT_DIR
data_folder = Path(models_folder_name)
list_model_files = list(data_folder.glob('*.bin'))
num_labels = 9

f1_score_list = []
for model_file in list_model_files:
    try:
        print("\n Model file: ", model_file)
        model = torch.load(model_file)
#         model = nn.DataParallel(model)
        x =model.eval()
        test_inputs, test_preds, test_labels, test_loss = eval_model( config, model, test_dataloader, device, num_labels)  
        print(classification_report(test_labels,test_preds ))
        print("\n \n \n")
        f1_value = f1_score(test_labels, test_preds,average='macro')
        f1_score_list.append([model_file,f1_value ])
        file_name = model_file.__str__().replace(".", "_") 
        save_missed_cases_to_file( file_name, test_preds, test_labels, test_inputs)
        
    except:
#         print("Failed due to cpu & GPU incompatibility")
     # printing stack trace 
        traceback.print_exc()         

print("\n \n \n")





 Model file:  outputs/re/BioBERT_5epochs.bin



              precision    recall  f1-score   support

           0       0.95      0.95      0.95      3464
           1       0.98      0.96      0.97      3723
           2       0.90      0.95      0.93      4274
           3       0.97      0.94      0.95      4090
           4       0.94      0.91      0.93       420
           5       0.97      0.91      0.94      3024
           6       0.98      0.97      0.97      2802
           7       0.92      0.82      0.87       775
           8       0.99      1.00      0.99     27438

    accuracy                           0.97     50010
   macro avg       0.95      0.93      0.94     50010
weighted avg       0.97      0.97      0.97     50010


 
 



INFO:filelock:Lock 140373291001392 acquired on /root/.cache/torch/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791.lock


INFO:filelock:Lock 140373291001392 released on /root/.cache/torch/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791.lock




 Model file:  outputs/re/BioBERT_5epochs_256seq.bin


...

Traceback (most recent call last):
  File "<ipython-input-9-c3208afc4473>", line 13, in <module>
    test_inputs, test_preds, test_labels, test_loss = eval_model( config, model, test_dataloader, device, num_labels)
  File "/content/drive/.shortcut-targets-by-id/1I3W7Z7rz_YfsjBjX7z4zTzGbr-iKTrmD/w266_final/project_re/eval.py", line 38, in eval_model
    logits = model(input_ids, segment_ids, input_mask)
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/content/drive/.shortcut-targets-by-id/1I3W7Z7rz_YfsjBjX7z4zTzGbr-iKTrmD/w266_final/project_re/model/MedClinical.py", line 76, in forward
    x1 = self.conv_1(x)
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/conv.py", line 259, in forward
    self.padding, self.dilation, self.gro


 Model file:  outputs/re/BioBERT_fc2020-11-25 06_24_08_491859.bin



              precision    recall  f1-score   support

           0       0.95      0.95      0.95      3464
           1       0.98      0.96      0.97      3723
           2       0.93      0.94      0.94      4274
           3       0.95      0.94      0.95      4090
           4       0.95      0.91      0.93       420
           5       0.97      0.92      0.94      3024
           6       0.98      0.97      0.97      2802
           7       0.91      0.84      0.87       775
           8       0.99      1.00      0.99     27438

    accuracy                           0.97     50010
   macro avg       0.96      0.94      0.95     50010
weighted avg       0.97      0.97      0.97     50010


 
 


 Model file:  outputs/re/BioBERT_fc2020-11-25 07_33_41_117403.bin



              precision    recall  f1-score   support

           0       0.95      0.95      0.95      3464
           1       0.97      0.96      0.96      3723
           2       0.92      0.95      0.93      4274
           3       0.96      0.94      0.95      4090
           4       0.95      0.90      0.93       420
           5       0.97      0.91      0.94      3024
           6       0.97      0.97      0.97      2802
           7       0.91      0.84      0.87       775
           8       0.99      1.00      0.99     27438

    accuracy                           0.97     50010
   macro avg       0.95      0.93      0.94     50010
weighted avg       0.97      0.97      0.97     50010


 
 


 Model file:  outputs/re/BioBERT_fc2020-11-25 17_16_55_635505.bin



              precision    recall  f1-score   support

           0       0.96      0.94      0.95      3464
           1       0.97      0.96      0.96      3723
           2       0.99      0.91      0.95      4274
           3       0.99      0.93      0.96      4090
           4       0.95      0.92      0.93       420
           5       0.98      0.91      0.94      3024
           6       0.94      0.97      0.95      2802
           7       0.91      0.84      0.87       775
           8       0.97      1.00      0.98     27438

    accuracy                           0.97     50010
   macro avg       0.96      0.93      0.95     50010
weighted avg       0.97      0.97      0.97     50010


 
 


 
 



In [10]:
pd.set_option('display.max_colwidth', 120)
columns = ['file_namae', 'f1-score']
test_results= pd.DataFrame(f1_score_list, columns = columns)
test_results

,file_namae,f1-score
0,outputs/re/BioBERT_5epochs.bin,0.944320
1,outputs/re/BioBERT_5epochs_256seq.bin,0.944499
2,outputs/re/BioBERT_fc2020-11-21 23_33_17_281822.bin,0.946691
3,outputs/re/BioBERT_fc2020-11-25 06_24_08_491859.bin,0.945477
4,outputs/re/BioBERT_fc2020-11-25 07_33_41_117403.bin,0.943868
5,outputs/re/BioBERT_fc2020-11-25 17_16_55_635505.bin,0.945282


In [11]:
# print ("F1 Score summary \n")  
# ax = plt.gca()
# ax.get_xaxis().set_visible(False)
# ax.get_yaxis().set_visible(False)
# plt.box(on=None)
# plt.figure(figsize=(14,6))
# the_table = plt.table(cellText=f1_score_list,
#                       colLabels=['file name', 'F1-score'],
#                       loc='center')
# the_table.auto_set_font_size(False)
# the_table.set_fontsize(10)